In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, MaxPool2D

from sklearn.model_selection import train_test_split


In [2]:
#Load the data
(X_train_unscaled, y_train), (X_test_unscaled, y_test) = cifar10.load_data()

print(f"X_train shape: {np.shape(X_train_unscaled)}")
print(f"y_train shape: {np.shape(y_train)}")
print(f"X_test shape: {np.shape(X_test_unscaled)}")
print(f"y_test shape: {np.shape(y_test)}")

X_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
X_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 1)


In [3]:
#scale the normalize the data and convert targets to one hot encoding
X_train_valid, X_test = X_train_unscaled/255.0, X_test_unscaled/255.0

y_cat_train_valid = to_categorical(y_train)
y_cat_test = to_categorical(y_test)

#split the training dataset again to produce a validation set

# Split the data into training and testing sets
X_train, X_valid, y_cat_train, y_cat_valid = train_test_split(X_train_valid, y_cat_train_valid, test_size=0.25, random_state=42)


In [4]:
#build the model
INPUT_SHAPE = (32,32,3)
KERNEL_SIZE = (3,3)
model = Sequential()

#common to increase number of filters as we add convolutional layers (pooling reduces by 2 so we double filters)

#conv layers
model.add(Conv2D(filters=32, kernel_size = KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size = KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
#pooling
model.add(MaxPool2D(pool_size=(2,2)))
#droput
model.add(Dropout(0.25))

#conv layers
model.add(Conv2D(filters=64, kernel_size = KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size = KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
#pooling
model.add(MaxPool2D(pool_size=(2,2)))
#droput
model.add(Dropout(0.25))

#conv layers
model.add(Conv2D(filters=128, kernel_size = KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size = KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))

model.add(BatchNormalization())
#pooling
model.add(MaxPool2D(pool_size=(2,2)))
#droput
model.add(Dropout(0.25))

#fully connected layers
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))

METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=METRICS)

In [5]:
#show the summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                        

In [6]:
#train the model
num_epochs = 20
model.fit(X_train, y_cat_train, epochs=num_epochs, validation_data=(X_valid,y_cat_valid), batch_size=32)

Epoch 1/20
1172/1172 [==============================] - 319s 271ms/step - loss: 1.7371 - accuracy: 0.3651 - precision: 0.6125 - recall: 0.1561 - val_loss: 2.5544 - val_accuracy: 0.2983 - val_precision: 0.3218 - val_recall: 0.1892
Epoch 2/20
1172/1172 [==============================] - 290s 248ms/step - loss: 1.2739 - accuracy: 0.5465 - precision: 0.7246 - recall: 0.3714 - val_loss: 1.1962 - val_accuracy: 0.5804 - val_precision: 0.7235 - val_recall: 0.4617
Epoch 3/20
1172/1172 [==============================] - 89s 75ms/step - loss: 1.0626 - accuracy: 0.6318 - precision: 0.7738 - recall: 0.4905 - val_loss: 0.8947 - val_accuracy: 0.6810 - val_precision: 0.7973 - val_recall: 0.5743
Epoch 4/20
1172/1172 [==============================] - 221s 189ms/step - loss: 0.9317 - accuracy: 0.6786 - precision: 0.7984 - recall: 0.5625 - val_loss: 0.8185 - val_accuracy: 0.7190 - val_precision: 0.8055 - val_recall: 0.6351
Epoch 5/20
1172/1172 [==============================] - 434s 370ms/step - loss: 0.

In [13]:
#evaluate the model on the test set
test_results = model.evaluate(X_test, y_cat_test)
loss = test_results[0]
accuracy = test_results[1]
precision = test_results[2]
recall = test_results[3]

print(f'accuracy: {accuracy}')
print(f'precision: {precision}')
print(f'recall: {recall}')

313/313 [==============================] - 4s 13ms/step - loss: 0.6093 - accuracy: 0.8081 - precision: 0.8456 - recall: 0.7807
accuracy: 0.8080999851226807
precision: 0.845553994178772
recall: 0.7807000279426575
